# ¡Día de Gradio!

Hoy crearemos interfaces de usuario utilizando el increíblemente simple marco Gradio.



In [ ]:
# imports

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic

In [ ]:
import gradio as gr # oh yeah!

In [ ]:
# Cargar variables de entorno en un archivo llamado .env
# Imprimir los prefijos de clave para facilitar la depuración

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')  # OpenRouter maneja todos los modelos

# Usar la misma API key para todas las peticiones
openai_api_key = api_key


In [ ]:
# Crear una instancia de OpenAI
openai =OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=openai_api_key
)

In [ ]:
# Un mensaje genérico del sistema

system_message = "Eres un asistente útil"

In [ ]:
# Envolvamos una llamada a GPT-4o-mini en una función simple

def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = openai.chat.completions.create(
        model="openai/gpt-4o-mini",
        messages=messages,
    )
    return completion.choices[0].message.content

In [ ]:
message_gpt("¿Cual es la fecha de hoy?")

## Momento de crear nuestra interfaz de usuario

In [ ]:
# Aquí hay una función simple

def shout(text):
    print(f"Has llamado a la función shout con el mensaje: {text}")
    return text.upper()

In [ ]:
shout("Hola")

In [ ]:
# Crear una interfaz de usuario simple

gr.Interface(fn=shout, inputs="textbox", outputs="textbox").launch()

In [ ]:
# Agregar share=True significa que se puede acceder a él de forma pública

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True)

In [ ]:
# Y ahora, cambiando la función de "shout" a "message_gpt"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Tu Mensaje:", lines=6)],
    outputs=[gr.Textbox(label="Respuesta:", lines=8)],
    flagging_mode="never"
)
view.launch(share=True)

In [ ]:
# Usemos Markdown


system_message = "Eres una asistente útil que responde en formato markdown"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Tu mensaje:")],
    outputs=[gr.Markdown(label="Respuesta:")],
    flagging_mode="never"
)
view.launch()

In [ ]:
# Vamos a crear una llamada que transmita resultados



def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Tu :")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

In [ ]:
#USAR LA MISMA BASE SOLO CAMBIANDO EL VALOR DEL MODELO POR EL MODELO DESEADO, SACADO DE OPENROUTER
def stream_claude(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model="anthropic/claude-3.5-haiku-20241022:beta",
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
view = gr.Interface(
    fn=stream_claude, #SE PASA LA FUNCION CREADA ARRIBA stream_claude
    inputs=[gr.Textbox(label="Tu :")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

## Una mejora menor

He realizado una pequeña mejora en este código.

Anteriormente, tenía estas líneas:

```
for chunk in result:
yield chunk
```

En realidad, hay una forma más elegante de lograr esto (que los expertos en Python podrían llamar más "Pythonico"):

`yield from result`

Abordo esto con más detalle en el cuaderno de Python intermedio en la carpeta week1; échale un vistazo si quieres más información.

In [ ]:
def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Modelo Desconocido")
    yield from result

In [ ]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Tu mensaje:"), gr.Dropdown(["GPT", "Claude"], label="Selecciona un modelo:", value="GPT")],
    outputs=[gr.Markdown(label="Respuesta:")],
    flagging_mode="never"
)
view.launch()

# Cómo crear un generador de folletos de empresa


In [ ]:
# Una clase para representar una página web

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No se ha encontrado título de la página"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Título de la Web:\n{self.title}\nContenido de la Web:\n{self.text}\n\n"

In [ ]:


system_message = "Eres un asistente que analiza el contenido de la página de inicio \
del sitio web de una empresa y crea un folleto breve sobre la empresa para posibles clientes, inversores y nuevos empleados.\
Responde en formato Markdown."

In [ ]:
def stream_brochure(company_name, url, model):
    prompt = f"Genera un folleto de la empresa {company_name}. Esta es su página de destino:\n"    
    prompt += Website(url).get_contents()
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Modelo Desconocido")
    yield from result

In [ ]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Nombre de la Empresa:"),
        gr.Textbox(label="Landing page, recuerda incluir http:// o https://"),
        gr.Dropdown(["GPT", "Claude"], label="Selecciona un modelo")],
    outputs=[gr.Markdown(label="Folleto:")],
    flagging_mode="never"
)
view.launch()